### Context Manager 環境管理器

這一個章節的內容主要來自于 "Programming in Python3" 這本書，

加上我一些說明，希望有更清楚一些。

![](http://www.qtrac.eu/py3bookm.png)

<center>
(這是一本很囉唆的好書，如果你喜歡研究一些細節的東西，蠻建議買來看看)
</center>

----

什麼是 context manager ?

在 python 裡，如果你有一個 context manager 例如說 cm ，

你就可以使用 `with` 陳述句例如：

    with cm as f:
        """
        do something with f ...
        """
        ...

簡單的來說，一個環境管理器就是有實做 `__enter__` 與 `__exit__` 兩個特殊方法的物件。

以最常見的檔案物件來說:

In [3]:
file = open("sample.txt", "r")

In [10]:
"__enter__" in dir(file) and "__exit__" in dir(file) # 的確有 __enter__ 與 __exit__

True

In [11]:
file.close() # 嗯....每次都要記得 close 真麻煩。

科技始終來自于人性，人性總是不離惰性。

藉由 `__enter__` 與 `__exit__` 你可以有效率地管理這些事情。(開一個檔案、關檔案...etc)

In [12]:
# 使用 with 
with open("sample.txt", "r") as file:
    lyrics = file.read()
    print(lyrics)


如果還有明天

作詞：劉偉仁
作曲：劉偉仁

我們都有看不開的時候　總有冷落自己的舉動
但是我一定會提醒自己　如果還有明天

我們都有傷心的時候　總不在乎這種感受
但是我要把握每次感動　如果還有明天

＊如果還有明天　你想怎樣裝扮你的臉
　如果沒有明天　要怎麼說再見

如果你看出我的遲疑　是不是你也想要問我
究竟有多少事還沒做　如果還有明天

如果真的還能夠有明天　是否能把事情都做完
是否一切也將雲消煙散　如果沒有明天

Repeat ＊,＊,＊




In [14]:
file.closed # 正確地被關閉了!

True

基本上，要實做一個環境管理器並不難，大致上要注意這幾點:

1. `__enter__` 傳回的對象會指定給 `with` 陳述句後 `as` 的變數。( `with xxx as 變數: # <-- 就是它!`)
2. 無論有沒有發生例外，`__exit__` 都會被執行。
3. 例外會被捕捉並以參數的方式傳給 `__exit__`，你可以在 `__exit__` 中對例外做額外的處理。
4. 若 `__exit__` 回傳的值為 True，則任何例外都不會向外傳遞，反之則向外傳遞。

In [22]:
## 實做一個看看吧!

class MyOpenAsList:
    
    def __init__(self, filename, mode = None):
        if not mode:
            mode = "r"
        self.__file = open(filename, mode)
    
    def __enter__(self):
        print("Opening file!")
        content = self.__file.readlines()
        return content
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        exc_type: Exception type object (就是一個 class 啦!)
        exc_val: 該 Exception 的 value
        exc_tb: 一個 traceback object
        """
        print("exc_type: ", exc_type)
        print("exc_val:", exc_val)
        print("exc_tb: ", exc_tb)
        print("Clossing file!")
        self.__file.close()
        # 沒有回傳值，所以 MyOpenAsList 捕捉到的例外會向外傳遞。

In [23]:
with MyOpenAsList("sample.txt") as content:
    print("".join(content))

Opening file!

如果還有明天

作詞：劉偉仁
作曲：劉偉仁

我們都有看不開的時候　總有冷落自己的舉動
但是我一定會提醒自己　如果還有明天

我們都有傷心的時候　總不在乎這種感受
但是我要把握每次感動　如果還有明天

＊如果還有明天　你想怎樣裝扮你的臉
　如果沒有明天　要怎麼說再見

如果你看出我的遲疑　是不是你也想要問我
究竟有多少事還沒做　如果還有明天

如果真的還能夠有明天　是否能把事情都做完
是否一切也將雲消煙散　如果沒有明天

Repeat ＊,＊,＊


exc_type:  None
exc_val: None
exc_tb:  None
Clossing file!


In [25]:
with MyOpenAsList("sample.txt") as file:
    raise OSError("can't open file")

Opening file!
exc_type:  <class 'OSError'>
exc_val: can't open file
exc_tb:  <traceback object at 0x107c3c788>
Clossing file!


OSError: can't open file

這樣就簡單的實做了一個環境管理器。

至於我說的所謂例外向外傳遞的這件事，

我們用下面這個環境管理器來說明:

In [33]:
class MyOpenAsListSilent:
    
    def __init__(self, filename, mode = "r", silent = True):
        self.silent = silent
        self.filename = filename
        try:
            self.__file = open(filename, mode)
        except Exception as e:
            if not silent:
                raise e
    
    def __enter__(self):
        if getattr(self, "_MyOpenAsListSilent__file", None):
            return self.__file.readlines()
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        if self.silent:
            return True
        if getattr(self, "_MyOpenAsListSilent__file", None):
            self.__file.close()

In [36]:
with MyOpenAsListSilent("sample.txt") as file:
    raise Exception("Bad thing happend!")

with MyOpenAsListSilent("sample.txt") as file:
    raise IOError("Can't open file!")

所有 Exception 都不見了!!

原因是因為這時候我們的 `__exit__` 都是回傳 `True`，這也就是剛剛說的不會向外傳遞的意思。

In [37]:
with MyOpenAsListSilent("sample.txt", silent=False) as file:
    raise IOError("Can't open file!")

OSError: Can't open file!

至於為什麼現在又可以了....自己想想吧! 不難噢~